In [8]:
import numpy as np

'Train'

In [1]:
%cd ../

/home/zhangtianning.di/projects_local/PromptNougat


In [2]:
from model.model_arguements import FlougatSmallConfig,UpArouGatSmallConfig
from model.model_arguements import TrainerModelConfig
from model.trainer_model import PromptNougatModel, LlougatTrainerModel
from model.replace_flash_attn import replace_promptdecoder_attn_with_flash_attn

In [3]:
from dataclasses import dataclass
from simple_parsing import ArgumentParser

parser = ArgumentParser()
parser.add_arguments(UpArouGatSmallConfig, dest="config")
config = parser.parse_args(["--attn_implementation","eager"]).config

In [4]:
model =  LlougatTrainerModel(config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import torch

In [6]:
batch = torch.load("input.pt",map_location='cpu')

In [7]:
model(**batch)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


(tensor(nan, grad_fn=<DivBackward0>),
 {'start_token_loss': tensor(nan, grad_fn=<NllLossBackward0>),
  'start_bbox_diou': tensor(nan, grad_fn=<DivBackward0>),
  'start_bbox_iou': tensor(nan, grad_fn=<DivBackward0>),
  'token_loss': tensor(nan, grad_fn=<NllLossBackward0>),
  'bbox_diou': tensor(nan, grad_fn=<DivBackward0>),
  'bbox_iou': tensor(nan, grad_fn=<DivBackward0>)},
 (tensor([[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]],
  
          [[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]

In [11]:
4* 4096* 16* 48

12582912

In [10]:
import torch
old_weight = torch.load("pretrain_weight/models--microsoft--Florence-2-base-ft/snapshots/e7a5acc73559546de6e12ec0319cd7cc1fa2437c/pytorch_model.bin")

In [ ]:
now_keys = set(model.state_dict().keys())

In [15]:
state_dict = model.state_dict()

In [17]:
for key in old_weight.keys():
    vision_key = "encoder."+key
    if vision_key in state_dict:
        state_dict[vision_key] = old_weight[key]

In [18]:
torch.save(state_dict, "pretrain_weight/FlougatSmall_start.model.bin")

In [2]:
from decoder.llougat.modeling_llougat import *
hidden_size = 32
config = LlougatConfig(image_size = [896,672], image_embedding_size=[28, 21],num_hidden_layers=2,
                       num_attention_heads=16,num_key_value_heads=16,intermediate_size=1024,
                       hidden_size=hidden_size, _attn_implementation='flash_attention_2')
decoder = LlougatForCausalLM(config)

Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlougatForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlougatModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


In [18]:
decoder = decoder.cuda()
encoder_hidden_states  = torch.randn(2, 28*21, hidden_size)
input_ids    = torch.randint(0,32,(2, 10))
input_bboxes = torch.randn((2, 10, 4))
input_ids[0][5:]=0
input_bboxes[0][5:]*=0

In [3]:
%cd ../

/data/ztn/projects/PromptNougat


/root/miniconda3/envs/tntorch/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import torch
batch = torch.load('input.pt')

In [10]:
batch['pre_input_ids']

tensor([[ 0,  0, 42,  ...,  1,  1,  1],
        [ 0,  0, 69,  ...,  1,  1,  1]], device='cuda:0')

In [8]:
batch['prompt_in'][0]

tensor([[[0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.1728, 0.1565],
         [0.2193, 0.1707]],

        [[0.1728, 0.1565],
         [0.2193, 0.1707]],

        ...,

        [[0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.0000, 0.0000],
         [0.0000, 0.0000]],

        [[0.0000, 0.0000],
         [0.0000, 0.0000]]], device='cuda:0')

In [ ]:
result = decoder()

# Unit Test

In [4]:
from model.decoder.llougat.modeling_llougat import *
hidden_size = 32
config = LlougatConfig(image_size = [896,672], image_embedding_size=[28, 21],num_hidden_layers=2,
                       num_attention_heads=16,num_key_value_heads=16,intermediate_size=1024,
                       hidden_size=hidden_size, _attn_implementation='eager',coordinate_retreive_method="heatmap_mapping")
decoder = LlougatForCausalLM(config)

In [3]:
from model.decoder.locr.modeling_locr import *
hidden_size = 128
config = PromptBartConfig(image_size = [896,672], image_embedding_size=[28, 21],num_hidden_layers=2,
                         num_attention_heads=16,num_key_value_heads=16,intermediate_size=1024,
                         hidden_size=hidden_size,BartAttention_implement='flashattn')
decoder = PromptBartForCausalLM(config).eval()

use position decay == 1, please note it is not the best setting for generation


In [4]:
decoder = decoder.cuda()
encoder_hidden_states  = torch.randn(2, 28*21, hidden_size)
input_ids    = torch.randint(0,hidden_size,(2, 10))
input_bboxes = torch.randn((2, 10, 4))
input_ids[0][5:]=0
input_bboxes[0][5:]*=0

In [5]:
self = decoder
hidden_states = inputs_embeds  = decoder.decoder.embed_tokens(input_ids.cuda()).cuda()
encoder_hidden_states = torch.randn(2, 28*21, hidden_size).cuda()
attention_mask = torch.ones_like(input_ids)
attention_mask[0][5:]=0
attention_mask = attention_mask.cuda()
B, S, _ = hidden_states.shape
B, L, _ = encoder_hidden_states.shape
self = decoder

In [55]:
cache_position = torch.arange(0, 0 + inputs_embeds.shape[1], device=inputs_embeds.device)
position_ids = cache_position.unsqueeze(0)

In [6]:
from model.decoder.locr.modeling_locr import PromptAttentionFlash,PromptAttention,_prepare_4d_attention_mask

In [7]:
input_shape = input_ids.shape

In [8]:
encoder_hidden_states = torch.randn(2, 28*21, 1024).cuda()

In [11]:
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.float16):
        decoder.decoder.config.BartAttention_implement = 'flashattn'
        causal_mask = decoder.decoder._prepare_decoder_attention_mask(
            attention_mask, None, None, input_shape, inputs_embeds,0
        )
        result1= MBartFlashAttention2.forward(decoder.decoder.layers[0].self_attn,
                hidden_states    = hidden_states.cuda(),              
                attention_mask   = causal_mask.cuda(),   
            )[0]
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.float16):
        decoder.decoder.config.BartAttention_implement = 'eager'
        causal_mask = decoder.decoder._prepare_decoder_attention_mask(
            attention_mask, None, None, input_shape, inputs_embeds,0
        )
        result2= MBartAttention.forward(decoder.decoder.layers[0].self_attn,
                hidden_states    = hidden_states.cuda(),              
                attention_mask   = causal_mask.cuda(),   
            )[0]
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.float16):
        decoder.decoder.config.BartAttention_implement = 'sdpa'
        causal_mask = decoder.decoder._prepare_decoder_attention_mask(
            attention_mask, None, None, input_shape, inputs_embeds,0
        )
        result3= MBartAttentionSDPA.forward(decoder.decoder.layers[0].self_attn,
                hidden_states    = hidden_states.cuda(),              
                attention_mask   = causal_mask.cuda(),   
            )[0]

In [12]:
attention_mask = attention_mask.bool()

In [13]:
torch.dist(result1[attention_mask],result2[attention_mask])

tensor(0.0200, device='cuda:0', dtype=torch.float16)

In [14]:
torch.dist(result1[attention_mask],result3[attention_mask])

tensor(0.0200, device='cuda:0', dtype=torch.float16)

In [10]:
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        decoder.model.config._attn_implementation = 'eager'
        cross_attn_mask = decoder.model.get_cross_attn_mask(attention_mask, inputs_embeds,  encoder_hidden_states)
        result1= LlougatCrossAttention.forward(decoder.model.layers[0].cross_attn,
                hidden_states    = hidden_states.cuda(),              
                hidden_states_kv = encoder_hidden_states.cuda(),
                attention_mask   = cross_attn_mask.cuda()
            )[0]

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        decoder.model.config._attn_implementation = 'flash_attention_2'
        cross_attn_mask = decoder.model.get_cross_attn_mask(attention_mask, inputs_embeds,  encoder_hidden_states)
        result2= LlougatCrossFlashAttention2.forward(decoder.model.layers[0].cross_attn,
                hidden_states    = hidden_states.cuda(),              
                hidden_states_kv = encoder_hidden_states.cuda(),
                attention_mask   = cross_attn_mask.cuda()
            )[0]

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        decoder.model.config._attn_implementation = 'sdpa'
        cross_attn_mask = decoder.model.get_cross_attn_mask(attention_mask, inputs_embeds,  encoder_hidden_states)
        result3= LlougatCrossSdpaAttention.forward(decoder.model.layers[0].cross_attn,
                hidden_states    = hidden_states.cuda(),              
                hidden_states_kv = encoder_hidden_states.cuda(),
                attention_mask   = cross_attn_mask.cuda()
            )[0]

AttributeError: 'PromptBartForCausalLM' object has no attribute 'model'

In [29]:
torch.dist(result1[attention_mask],result3[attention_mask])

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [7]:
coordinate = torch.randn(2, 28*21, 4).cuda()

In [13]:
%%timeit
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        decoder.model.config._attn_implementation = 'eager'
        cross_attn_mask = decoder.model.get_cross_attn_mask(attention_mask, inputs_embeds,  encoder_hidden_states)
        result1= LlougatCrossAttention.forward(decoder.position_attn,
                hidden_states    = hidden_states.cuda(),              
                hidden_states_kv =(encoder_hidden_states.cuda(), coordinate),
                attention_mask   = cross_attn_mask.cuda()
            )[0]

243 μs ± 315 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        decoder.model.config._attn_implementation = 'sdpa'
        cross_attn_mask = decoder.model.get_cross_attn_mask(attention_mask, inputs_embeds,  encoder_hidden_states)
        result3= LlougatCrossSdpaAttention.forward(decoder.position_attn,
                hidden_states    = hidden_states.cuda(),              
                hidden_states_kv =(encoder_hidden_states.cuda(),coordinate ),
                attention_mask   = cross_attn_mask.cuda()
            )[0]

282 μs ± 1.34 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
torch.dist(result1[attention_mask],result3[attention_mask])

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

# Example Usage:
query, key, value = torch.randn(2, 3, 8, device=device), torch.randn(2, 3, 8, device=device), torch.randn(2, 3, 8, device=device)

import torch.utils.benchmark as benchmark
def benchmark_torch_function_in_microseconds(f, *args, **kwargs):
    t0 = benchmark.Timer(
        stmt="f(*args, **kwargs)", globals={"args": args, "kwargs": kwargs, "f": f}
    )
    return t0.blocked_autorange().mean * 1e6

# Lets define the hyper-parameters of our input
batch_size = 32
max_sequence_len = 1024
num_heads = 32
embed_dimension = 32

dtype = torch.float16

query = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
key = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)
value = torch.rand(batch_size, num_heads, max_sequence_len, embed_dimension, device=device, dtype=dtype)

print(f"The default implementation runs in {benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value):.3f} microseconds")

# Lets explore the speed of each of the 3 implementations
from torch.nn.attention import SDPBackend, sdpa_kernel


with sdpa_kernel(SDPBackend.MATH):
    math_time=benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value)
    print(f"The math implementation runs in {math_time:.3f} microseconds")

with sdpa_kernel(SDPBackend.FLASH_ATTENTION):
    try:
        flash_time=benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value)
        print(f"The flash attention implementation runs in {flash_time:.3f} microseconds")
    except RuntimeError:
        print("FlashAttention is not supported. See warnings for reasons.")

with sdpa_kernel(SDPBackend.EFFICIENT_ATTENTION):
    try:
        efficient_time=benchmark_torch_function_in_microseconds(F.scaled_dot_product_attention, query, key, value)
        print(f"The memory efficient implementation runs in {efficient_time:.3f} microseconds")
    except RuntimeError:
        print("EfficientAttention is not supported. See warnings for reasons.")

The default implementation runs in 666.268 microseconds
The math implementation runs in 3759.861 microseconds
The flash attention implementation runs in 665.455 microseconds
The memory efficient implementation runs in 1816.581 microseconds


# Flash attn native test

In [22]:
import math
import torch
import torch.nn.functional as F
from einops import rearrange, repeat
from flash_attn import (
    flash_attn_func,
    flash_attn_kvpacked_func,
    flash_attn_qkvpacked_func,
    flash_attn_varlen_func,
    flash_attn_varlen_kvpacked_func,
    flash_attn_varlen_qkvpacked_func,
    flash_attn_with_kvcache,
)
from flash_attn.bert_padding import pad_input, unpad_input
from flash_attn.layers.rotary import apply_rotary_emb


In [23]:
def attention_ref(
    q,
    k,
    v,
    query_padding_mask=None,
    key_padding_mask=None,
    dropout_p=0.0,
    dropout_mask=None,
    causal=False,
    window_size=(-1, -1),  # -1 means infinite window size
    upcast=True,
    reorder_ops=False,
):
    """
    Arguments:
        q: (batch_size, seqlen_q, nheads, head_dim)
        k: (batch_size, seqlen_k, nheads_k, head_dim)
        v: (batch_size, seqlen_k, nheads_k, head_dim)
        query_padding_mask: (batch_size, seqlen_q)
        key_padding_mask: (batch_size, seqlen_k)
        dropout_p: float
        dropout_mask: (batch_size, nheads, seqlen_q, seqlen_k)
        causal: whether to apply causal masking
        window_size: (int, int), left and right window size
        upcast: whether to cast all inputs to fp32, do all computation in fp32, then cast
            output back to fp16/bf16.
        reorder_ops: whether to change the order of operations (scaling k instead of scaling k, etc.)
            without changing the math. This is to estimate the numerical error from operation
            reordering.
    Output:
        output: (batch_size, seqlen_q, nheads, head_dim)
        attention: (batch_size, nheads, seqlen_q, seqlen_k), softmax after dropout
    """
    if causal:
        window_size = (window_size[0], 0)
    dtype_og = q.dtype
    if upcast:
        q, k, v = q.float(), k.float(), v.float()
    seqlen_q, seqlen_k = q.shape[1], k.shape[1]
    k = repeat(k, "b s h d -> b s (h g) d", g=q.shape[2] // k.shape[2])
    v = repeat(v, "b s h d -> b s (h g) d", g=q.shape[2] // v.shape[2])
    d = q.shape[-1]
    if not reorder_ops:
        scores = torch.einsum("bthd,bshd->bhts", q / math.sqrt(d), k)
    else:
        scores = torch.einsum("bthd,bshd->bhts", q, k / math.sqrt(d))
    if key_padding_mask is not None:
        scores.masked_fill_(rearrange(~key_padding_mask, "b s -> b 1 1 s"), float("-inf"))
    if window_size[0] >= 0 or window_size[1] >= 0:
        local_mask = construct_local_mask(
            seqlen_q,
            seqlen_k,
            window_size,
            query_padding_mask,
            key_padding_mask,
            q.device,
        )
        scores.masked_fill_(local_mask, float("-inf"))
    attention = torch.softmax(scores, dim=-1)
    # Some rows might be completely masked out so we fill them with zero instead of NaN
    if window_size[0] >= 0 or window_size[1] >= 0:
        attention = attention.masked_fill(torch.all(local_mask, dim=-1, keepdim=True), 0.0)
    # We want to mask here so that the attention matrix doesn't have any NaNs
    # Otherwise we'll get NaN in dV
    if query_padding_mask is not None:
        attention = attention.masked_fill(rearrange(~query_padding_mask, "b s -> b 1 s 1"), 0.0)
    dropout_scaling = 1.0 / (1 - dropout_p)
    # attention_drop = attention.masked_fill(~dropout_mask, 0.0) * dropout_scaling
    # output = torch.einsum('bhts,bshd->bthd', attention_drop , v)
    if dropout_mask is not None:
        attention_drop = attention.masked_fill(~dropout_mask, 0.0)
    else:
        attention_drop = attention
    output = torch.einsum("bhts,bshd->bthd", attention_drop, v * dropout_scaling)
    if query_padding_mask is not None:
        output.masked_fill_(rearrange(~query_padding_mask, "b s -> b s 1 1"), 0.0)
    return output.to(dtype=dtype_og), attention.to(dtype=dtype_og)
def generate_qkv(
    q, k, v, query_padding_mask=None, key_padding_mask=None, kvpacked=False, qkvpacked=False
):
    """
    Arguments:
        q: (batch_size, seqlen_q, nheads, d)
        k: (batch_size, seqlen_k, nheads_k, d)
        v: (batch_size, seqlen_k, nheads_k, d)
        query_padding_mask: (batch_size, seqlen), bool
        key_padding_mask: (batch_size, seqlen), bool
    """
    assert not (kvpacked and qkvpacked)
    batch_size, seqlen_q, nheads, d = q.shape
    _, seqlen_k, nheads_k, _ = k.shape
    assert k.shape == (batch_size, seqlen_k, nheads_k, d)
    assert v.shape == (batch_size, seqlen_k, nheads_k, d)

    if query_padding_mask is not None:
        q_unpad, indices_q, cu_seqlens_q, max_seqlen_q = unpad_input(q, query_padding_mask)
        output_pad_fn = lambda output_unpad: pad_input(output_unpad, indices_q, batch_size, seqlen_q)
    else:
        q_unpad = rearrange(q, "b s h d -> (b s) h d")
        cu_seqlens_q = torch.arange(0, (batch_size + 1) * seqlen_q, step=seqlen_q, dtype=torch.int32, device=q_unpad.device)
        max_seqlen_q = seqlen_q
        output_pad_fn = lambda output_unpad: rearrange(output_unpad, "(b s) h d -> b s h d", b=batch_size)

    if key_padding_mask is not None:
        k_unpad, indices_k, cu_seqlens_k, max_seqlen_k = unpad_input(k, key_padding_mask)
        v_unpad, _, _, _ = unpad_input(v, key_padding_mask)
    else:
        k_unpad = rearrange(k, "b s h d -> (b s) h d")
        v_unpad = rearrange(v, "b s h d -> (b s) h d")
        cu_seqlens_k = torch.arange(0, (batch_size + 1) * seqlen_k, step=seqlen_k, dtype=torch.int32, device=k_unpad.device)
        max_seqlen_k = seqlen_k

    if qkvpacked:
        assert (query_padding_mask == key_padding_mask).all()
        assert nheads == nheads_k
        qkv_unpad = torch.stack([q_unpad, k_unpad, v_unpad], dim=1)
        qkv = torch.stack([q, k, v], dim=2)
        if query_padding_mask is not None:
            dqkv_pad_fn = lambda dqkv_unpad: pad_input(dqkv_unpad, indices_q, batch_size, seqlen_q)
        else:
            dqkv_pad_fn = lambda dqkv_unpad: rearrange(dqkv_unpad, "(b s) t h d -> b s t h d", b=batch_size)
        return (
            qkv_unpad.detach().requires_grad_(),
            cu_seqlens_q,
            max_seqlen_q,
            qkv.detach().requires_grad_(),
            output_pad_fn,
            dqkv_pad_fn,
        )
    elif kvpacked:
        kv_unpad = torch.stack([k_unpad, v_unpad], dim=1)
        kv = torch.stack([k, v], dim=2)
        dq_pad_fn = output_pad_fn
        if key_padding_mask is not None:
            dkv_pad_fn = lambda dkv_unpad: pad_input(dkv_unpad, indices_k, batch_size, seqlen_k)
        else:
            dkv_pad_fn = lambda dkv_unpad: rearrange(
                dkv_unpad, "(b s) t h d -> b s t h d", b=batch_size
            )
        return (
            q_unpad.detach().requires_grad_(),
            kv_unpad.detach().requires_grad_(),
            cu_seqlens_q,
            cu_seqlens_k,
            max_seqlen_q,
            max_seqlen_k,
            q.detach().requires_grad_(),
            kv.detach().requires_grad_(),
            output_pad_fn,
            dq_pad_fn,
            dkv_pad_fn,
        )
    else:
        dq_pad_fn = output_pad_fn
        if key_padding_mask is not None:
            dk_pad_fn = lambda dk_unpad: pad_input(dk_unpad, indices_k, batch_size, seqlen_k)
        else:
            dk_pad_fn = lambda dk_unpad: rearrange(dk_unpad, "(b s) h d -> b s h d", b=batch_size)
        return (
            q_unpad.detach().requires_grad_(),
            k_unpad.detach().requires_grad_(),
            v_unpad.detach().requires_grad_(),
            cu_seqlens_q,
            cu_seqlens_k,
            max_seqlen_q,
            max_seqlen_k,
            q.detach().requires_grad_(),
            k.detach().requires_grad_(),
            v.detach().requires_grad_(),
            output_pad_fn,
            dq_pad_fn,
            dk_pad_fn,
        )
def generate_random_padding_mask(max_seqlen, batch_size, device, mode="random"):
    assert mode in ["full", "random", "third"]
    if mode == "full":
        lengths = torch.full((batch_size, 1), max_seqlen, device=device, dtype=torch.int32)
    elif mode == "random":
        lengths = torch.randint(
            max(1, max_seqlen - 20), max_seqlen + 1, (batch_size, 1), device=device
        )
    elif mode == "third":
        lengths = torch.randint(max_seqlen // 3, max_seqlen + 1, (batch_size, 1), device=device)
    padding_mask = (
        repeat(torch.arange(max_seqlen, device=device), "s -> b s", b=batch_size) < lengths
    )
    return padding_mask


In [161]:
def _upad_input(query_layer, key_layer, value_layer, attention_mask, query_length):
        indices_k, cu_seqlens_k, max_seqlen_in_batch_k = _get_unpad_data(attention_mask)
        batch_size, kv_seq_len, num_key_value_heads, head_dim = key_layer.shape
        
        key_layer = index_first_axis(
            key_layer.reshape(batch_size * kv_seq_len, num_key_value_heads, head_dim), indices_k
        )
        value_layer = index_first_axis(
            value_layer.reshape(batch_size * kv_seq_len, num_key_value_heads, head_dim), indices_k
        )
        if query_length == kv_seq_len:
            num_heads = query_layer.shape[-2]
            query_layer = index_first_axis(
                query_layer.reshape(batch_size * kv_seq_len, self.num_heads, head_dim), indices_k
            )
            cu_seqlens_q = cu_seqlens_k
            max_seqlen_in_batch_q = max_seqlen_in_batch_k
            indices_q = indices_k
        elif query_length == 1:
            max_seqlen_in_batch_q = 1
            cu_seqlens_q = torch.arange(
                batch_size + 1, dtype=torch.int32, device=query_layer.device
            )  # There is a memcpy here, that is very bad.
            indices_q = cu_seqlens_q[:-1]
            query_layer = query_layer.squeeze(1)
        else:
            # The -q_len: slice assumes left padding.
            attention_mask = attention_mask[:, -query_length:]
            query_layer, indices_q, cu_seqlens_q, max_seqlen_in_batch_q = unpad_input(query_layer, attention_mask)

        return (
            query_layer,
            key_layer,
            value_layer,
            indices_q,
            (cu_seqlens_q, cu_seqlens_k),
            (max_seqlen_in_batch_q, max_seqlen_in_batch_k),
        )



In [ ]:
out_ref, attn_ref = attention_ref(
            q,
            k,
            v,
            query_padding_mask,
            key_padding_mask,
            dropout_p,
            dropout_mask,
            causal=causal,
            window_size=window_size,
        )

In [114]:
batch_size = 2
seqlen_q = 10
seqlen_k = 213
nheads = nheads_k = 16 
d = 32
device = "cuda"
dtype = torch.float16
q = torch.randn(batch_size, seqlen_q,   nheads, d, device=device, dtype=dtype, requires_grad=True)
k = torch.randn(batch_size, seqlen_k, nheads_k, d, device=device, dtype=dtype, requires_grad=True)
v = torch.randn(batch_size, seqlen_k, nheads_k, d, device=device, dtype=dtype, requires_grad=True)
query_padding_mask = generate_random_padding_mask(seqlen_q, batch_size, device, mode="random")
key_padding_mask = None
dropout_p = 0
causal = False
window_size=(-1, -1)

In [163]:
from transformers.models.llama.modeling_llama import LLAMA_ATTENTION_CLASSES, LlamaAttention, LlamaMLP, AttentionMaskConverter, Cache, LlamaRMSNorm, StaticCache, apply_rotary_pos_emb, repeat_kv, is_flash_attn_2_available, _get_unpad_data

In [174]:
query_layer,key_layer,value_layer,indices_q,(cu_seqlens_q, cu_seqlens_k),(max_seqlen_in_batch_q, max_seqlen_in_batch_k) =  _upad_input(q, k, v, query_padding_mask, seqlen_q)

In [176]:
k.shape

torch.Size([2, 213, 16, 32])

In [175]:
key_layer.shape

torch.Size([9, 16, 32])

In [168]:
torch.dist(key_layer,k_unpad)

RuntimeError: The size of tensor a (9) must match the size of tensor b (426) at non-singleton dimension 0

In [166]:
(
    q_unpad,
    k_unpad,
    v_unpad,
    cu_seqlens_q,
    cu_seqlens_k,
    max_seqlen_q,
    max_seqlen_k,
    q,
    k,
    v,
    output_pad_fn,
    dq_pad_fn,
    dk_pad_fn,
) = generate_qkv(q, k, v, query_padding_mask, key_padding_mask, kvpacked=False)

In [ ]:
out_unpad, sm_lse, S_dmask = flash_attn_varlen_func(
    q_unpad,
    k_unpad,
    v_unpad,
    cu_seqlens_q,
    cu_seqlens_k,
    max_seqlen_q,
    max_seqlen_k,
    dropout_p,
    return_attn_probs=True,
    causal=causal,
    window_size=window_size,
)
out = output_pad_fn(out_unpad)

In [116]:
out_ref, attn_ref = attention_ref(
    q,
    k,
    v,
    query_padding_mask,
    key_padding_mask,
    0.0,
    None,
    causal=causal,
    window_size=window_size,
)

In [120]:
query_states.shape

torch.Size([2, 10, 16, 32])

In [122]:
key_states.shape

torch.Size([2, 213, 16, 32])

In [123]:
decoder.model.config._attn_implementation = 'eager'
attention_mask = decoder.model.get_cross_attn_mask(query_padding_mask, query_states,  key_states)

In [124]:
attention_mask.shape

torch.Size([2, 1, 10, 213])

In [147]:
query_states = q
key_states   = k 
value_states = v
attn_weights = torch.einsum("bthd,bshd->bhts", query_states / math.sqrt(d), key_states)

In [131]:
causal_mask  = attention_mask[:, :, :, : key_states.shape[-2]] ##
causal_mask.shape

torch.Size([2, 1, 10, 16])

In [148]:
if attention_mask is not None:  # no matter the length, we just slice it
    causal_mask  = attention_mask[:, :, :, : key_states.shape[1]] ##
    attn_weights = attn_weights + causal_mask

In [157]:
# upcast attention to fp32
attn_weights = nn.functional.softmax(attn_weights, dim=-1)
attn_output  = torch.einsum("bhts,bshd->bthd", attn_weights, value_states)
bsz, q_len, _, _ = query_states.shape
bsz, v_len, H, d = value_states.shape

In [143]:
out_ref.shape

torch.Size([2, 10, 16, 32])

In [158]:
torch.dist(attn_output[query_padding_mask],out_ref[query_padding_mask])

tensor(0.0047, device='cuda:0', dtype=torch.float16, grad_fn=<DistBackward0>)

In [145]:
torch.dist(out,out_ref)

tensor(0.0021, device='cuda:0', dtype=torch.float16, grad_fn=<DistBackward0>)